In [1]:
!pip install pandas
!pip install numpy
!pip install pathlib
!pip install openpyxl

import pandas as pd
import numpy as np
from pathlib import Path

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
def summary_cleaning(summary_df, file_name):
    selected_columns1 = pd.DataFrame()
    values = summary_df["Unnamed: 1"].loc[3:5].values 
    selected_columns1 = pd.DataFrame([values], columns=["Brand","Locations", "City"])

    if summary_df["Unnamed: 1"].loc[6].startswith("Rest. ID - "):
       selected_columns1["Res-Id"] = int(summary_df["Unnamed: 1"].loc[6].split('-')[1])

    
    selected_columns2 = summary_df[["Unnamed: 1","Unnamed: 2"]].fillna('').loc[10:14].T.reset_index(drop = True) #loc index label name
    selected_columns2.columns = selected_columns2.iloc[0] #iloc postion of the row
    selected_columns2 = selected_columns2.drop(index=0).reset_index(drop = True)
    selected_columns2["Total Orders (Delivered + Cancelled)"] = selected_columns2["Total Orders (Delivered + Cancelled)"].astype(int)
    selected_columns2["Total Payout"] = selected_columns2["Total Payout"].str[1:].astype(float)
    selected_columns2["File Name"]= file_name

    return pd.concat([selected_columns1, selected_columns2] ,axis=1)

In [8]:
def payout_breakout_cleaning(payout_breakup_df, file_name, sr_no):
    payout_breakup_df.columns = payout_breakup_df.iloc[0].reset_index(drop=True)
    payout_breakup_df =  payout_breakup_df.drop(index=0).reset_index(drop = True)
    payout_breakup_df = payout_breakup_df.dropna(axis=1, how="all")
    payout_breakup_df.columns = payout_breakup_df.columns.where(~payout_breakup_df.columns.isna(), "SR.No")
    payout_breakup_df = payout_breakup_df.fillna('')
    payout_breakup_df["SR.No"] = sr_no + 1
    payout_breakup_df["File Name"] = file_name
    
    return payout_breakup_df



In [9]:
def order_level_cleaning(order_level_raw_df, file_name):
    order_df = order_level_raw_df.fillna('')
    order_df["File Name"] = file_name

    return order_df
    

In [11]:
folder = Path(r"C:\Users\jaian\Cleaning data & merging\Assignment files")

summary_rows = []                     
payout_breakout_rows = []
order_level_rows = []

for i, file in enumerate(folder.glob("*.xlsx")):
    
    summary_raw_df = pd.read_excel(file, sheet_name="Summary")
    summary = summary_cleaning(summary_raw_df, file.name) 
    
    payout_breakup_raw_df = pd.read_excel(file, sheet_name="Payout Breakup", skiprows = 1)
    payout_breakup = payout_breakout_cleaning(payout_breakup_raw_df, file.name, i)

    order_level_raw_df = pd.read_excel(file, sheet_name="Order Level", skiprows = 2)
    order_level = order_level_cleaning(order_level_raw_df, file.name)

    summary_rows.append(summary)    
    payout_breakout_rows.append(payout_breakup)
    order_level_rows.append(order_level)


summary_df = pd.concat(summary_rows, ignore_index=True)
payout_breakup_cleaned_df = pd.concat(payout_breakout_rows, ignore_index=True)
order_level_cleaned_df = pd.concat(order_level_rows, ignore_index=True)


payout_breakup_df = payout_breakup_cleaned_df.merge(summary_df[["Brand", "Res-Id", "Payout Period", "File Name"]], on="File Name")
cols = [col for col in payout_breakup_df.columns if col != "File Name"] + ["File Name"]
payout_breakup_df = payout_breakup_df[cols]


order_level_cleaned_df = summary_df[["Brand", "Res-Id", "Payout Period", "File Name"]].merge(order_level_cleaned_df, on="File Name")

commission_invoice_df = pd.read_excel(r"C:\Users\jaian\Cleaning data & merging\Swiggy_Tax_Sample_file.xlsx", sheet_name = "Swiggy.Tax")
commission_invoice_df = commission_invoice_df.iloc[:, 1:]
commission_invoice_cols = [commission_invoice_col for commission_invoice_col in commission_invoice_df.columns if commission_invoice_col != "id" and commission_invoice_col != "original_invoice_date"]
commission_invoice_df = commission_invoice_df[commission_invoice_cols]





In [13]:
with pd.ExcelWriter(r"C:\Users\jaian\Cleaning data & merging\Data Analyst Quick Assignment.xlsx",mode="a",engine="openpyxl",if_sheet_exists="overlay") as writer:
    summary_df[["Brand", "Locations", "City", "Res-Id", "Payout Period", "Payout Settlement Date", "Total Payout", "Total Orders (Delivered + Cancelled)", "Bank UTR"]].to_excel(writer, sheet_name = "Summary", index = False, startrow=1, header=False)


In [14]:
with pd.ExcelWriter(r"C:\Users\jaian\Cleaning data & merging\Data Analyst Quick Assignment.xlsx",mode="a",engine="openpyxl", if_sheet_exists="overlay") as writer:
    payout_breakup_df.to_excel(writer, sheet_name="Payout Breakup Tab", index=False, startrow=1, header=False)


In [15]:
with pd.ExcelWriter(r"C:\Users\jaian\Cleaning data & merging\Data Analyst Quick Assignment.xlsx",mode="a",engine="openpyxl", if_sheet_exists="overlay") as writer:
    order_level_cleaned_df.to_excel(writer, sheet_name="Order Level", index=False , startrow=1, header=False)

In [16]:
with pd.ExcelWriter(r"C:\Users\jaian\Cleaning data & merging\Data Analyst Quick Assignment.xlsx",mode="a",engine="openpyxl", if_sheet_exists="overlay") as writer:
    commission_invoice_df.to_excel(writer, sheet_name="Commission Invoice", index=False, startrow=1, header=False)

In [17]:
assi_df = pd.read_excel(r"C:\Users\jaian\Cleaning data & merging\Data Analyst Quick Assignment.xlsx", sheet_name="Summary")